In [2]:
import pyspark as ps
import pandas as pd
import matplotlib.pyplot as plt
from pyspark.sql.functions import udf, array
from pyspark.sql.types import StringType
from pyspark.sql.functions import col
from pyspark.sql.functions import countDistinct
import matplotlib.pyplot as plt
spark = (ps.sql.SparkSession.builder 
        .master("local[4]") 
        .appName("nathanscope") 
        .getOrCreate()
        )

In [3]:
sc = spark.sparkContext

In [4]:
def auth_casting_function(row):
    (time, scr_user, dst_user, src_comp, dst_comp, auth_type,logon_type,auth_orientation,success) = row
    feilds = [time, scr_user, dst_user, src_comp, dst_comp, auth_type,logon_type,auth_orientation,success]
    
    print('success')
    if(time=='?'):
        time = 666999666
    return (int(time),scr_user, dst_user, src_comp, dst_comp, auth_type,logon_type,auth_orientation,success)

In [5]:
def proc_casting_function(row):
    (time,user_domain,comp,proc_name,start) = row
    if(time=='?'):
        time = 666999666
    return (int(time),user_domain,comp,proc_name,start)

In [6]:
def flow_casting_function(row):
    (time, duration,src_comp,src_port,dst_comp,dst_port,protocol,pk_count,byte_count) = row
    if(time=='?'):
        time = 666999666
    return (int(time),int(duration),src_comp,src_port,dst_comp,dst_port,protocol,int(pk_count),int(byte_count))

In [7]:
def DNS_casting_function(row):
    (time, src_comp,comp_resolved) = row
    if(time=='?'):
        time = 666999666
    return (int(time),src_comp,comp_resolved)

In [8]:
# import the many data types
from pyspark.sql.types import *

# create a schema of your own
auth_schema = StructType( [
    StructField('time',     LongType(), True),
    StructField('src_user_domain',   StringType(),  True),
    StructField('dst_user_domain',   StringType(),  True),
    StructField('src_comp',   StringType(),  True),
    StructField('dst_comp',   StringType(),  True),
    StructField('auth_type',  StringType(), True),
    StructField('logon_type',  StringType(),  True),
    StructField('auth_orientation',  StringType(),  True),
    StructField('Success',  StringType(),  True)] )

proc_schema = StructType( [
    StructField('time',     LongType(), True),
    StructField('user_domain',   StringType(),  True),
    StructField('comp',   StringType(),  True),
    StructField('proc_name',   StringType(),  True),
    StructField('start',   StringType(),  True),
    ] )

flow_schema = StructType( [
    StructField('time',     LongType(), True),
    StructField('duration',     LongType(), True),
    StructField('src_comp',   StringType(),  True),
    StructField('src_port',   StringType(),  True),
    StructField('dst_comp',   StringType(),  True),
    StructField('dst_port',   StringType(),  True),
    StructField('protocol',  StringType(), True),
    StructField('pk_count',  LongType(),  True),
    StructField('byte_count',  LongType(),  True)] )

DNS_schema = StructType( [
    StructField('time',     LongType(), True),
    StructField('src_comp',   StringType(),  True),
    StructField('comp_resolved',   StringType(),  True),
    
    ] )

## Create RDDS from Text Input

rdd_auth = (sc.textFile('data/auth.txt')
            .map(lambda rowstr : rowstr.split(","))
            .map(auth_casting_function)
           )

rdd_proc = (sc.textFile('data/proc.txt')
            .map(lambda rowstr : rowstr.split(","))
            .map(proc_casting_function)
           )

rdd_flow = (sc.textFile('data/flows.txt')
            .map(lambda rowstr : rowstr.split(","))
            .map(flow_casting_function)
           )

rdd_DNS = (sc.textFile('data/dns.txt')
            .map(lambda rowstr : rowstr.split(","))
            .map(DNS_casting_function)
           )

## Create DataFrames from RDDs and 

In [9]:
auth_df = spark.createDataFrame(rdd_auth,auth_schema)

proc_df = spark.createDataFrame(rdd_proc,proc_schema)

flow_df = spark.createDataFrame(rdd_flow,flow_schema)

DNS_df = spark.createDataFrame(rdd_DNS,DNS_schema)

In [10]:
active_comps_df = pd.read_csv('Summary/Day1/active_comps_df.csv')
active_user_df= pd.read_csv('Summary/Day1/active_users_df.csv')
active_red_team =pd.read_csv('Summary/Day1/active_red_team.csv')

In [11]:
active_comps_df.columns = ['ignore','comp','count']

In [12]:
active_flow_df = flow_df.where(col("src_comp").isin(list(active_comps_df['comp'])) | col("dst_comp").isin(list(active_comps_df['comp'])))

red_flow_df = flow_df.where(col("src_comp").isin(list(active_red_team['dst_comp'])) | col("dst_comp").isin(list(active_red_team['dst_comp'])))




In [13]:
active_flow_df.collect()

Py4JJavaError: An error occurred while calling o3649.collectToPython.
: java.lang.OutOfMemoryError: GC overhead limit exceeded
	at org.apache.spark.sql.execution.SparkPlan$$anon$1.next(SparkPlan.scala:282)
	at org.apache.spark.sql.execution.SparkPlan$$anon$1.next(SparkPlan.scala:278)
	at scala.collection.Iterator$class.foreach(Iterator.scala:891)
	at org.apache.spark.sql.execution.SparkPlan$$anon$1.foreach(SparkPlan.scala:278)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$executeCollect$1.apply(SparkPlan.scala:300)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$executeCollect$1.apply(SparkPlan.scala:299)
	at scala.collection.IndexedSeqOptimized$class.foreach(IndexedSeqOptimized.scala:33)
	at scala.collection.mutable.ArrayOps$ofRef.foreach(ArrayOps.scala:186)
	at org.apache.spark.sql.execution.SparkPlan.executeCollect(SparkPlan.scala:299)
	at org.apache.spark.sql.Dataset$$anonfun$collectToPython$1.apply(Dataset.scala:3257)
	at org.apache.spark.sql.Dataset$$anonfun$collect

In [14]:
add_download = udf(lambda x : 1 if x in list(active_comps_df['comp']) else 0 )
add_upload = udf(lambda x : 1 if x in list(active_comps_df['comp']) else 0 )

In [15]:
active_flow_df = active_flow_df.withColumn('Download', add_download(active_flow_df['dst_comp']))
active_flow_df = active_flow_df.withColumn('Upload', add_upload(active_flow_df['src_comp']))

In [16]:
red_flow_df = red_flow_df.withColumn('Download', add_download(red_flow_df['dst_comp']))
red_flow_df = red_flow_df.withColumn('Upload', add_upload(red_flow_df['src_comp']))

In [17]:
add_count = udf(lambda x : x[1] if x[0]=='1' else 0 )

In [18]:
active_flow_df = active_flow_df.withColumn('upload_bytes',add_count(array('Upload','byte_count')))
active_flow_df = active_flow_df.withColumn('download_bytes',add_count(array('Download','byte_count')))
active_flow_df = active_flow_df.withColumn('upload_pk',add_count(array('Upload','pk_count')))
active_flow_df = active_flow_df.withColumn('download_pk',add_count(array('Download','pk_count')))

In [19]:
red_flow_df = red_flow_df.withColumn('upload_bytes',add_count(array('Upload','byte_count')))
red_flow_df = red_flow_df.withColumn('download_bytes',add_count(array('Download','byte_count')))
red_flow_df = red_flow_df.withColumn('upload_pk',add_count(array('Upload','pk_count')))
red_flow_df = red_flow_df.withColumn('download_pk',add_count(array('Download','pk_count')))

In [20]:
interval_size = 3600
make_interval = udf(lambda x :  (x//interval_size)*interval_size)

In [21]:
 active_flow_df = active_flow_df.withColumn('interval', make_interval(active_flow_df['time']))

In [22]:
red_flow_df = red_flow_df.withColumn('interval', make_interval(active_flow_df['time']))

In [ ]:
flow_time_series = active_flow_df.groupBy('interval').agg({'duration': 'mean','pk_count':'sum','byte_count':'sum','upload_bytes':'sum','download_bytes':'sum','upload_pk':'sum','download_pk':'sum'}).collect()



In [ ]:
red_flow_time_series = red_flow_df.groupBy('interval').agg({'duration': 'mean','pk_count':'sum','byte_count':'sum','upload_bytes':'sum','download_bytes':'sum','upload_pk':'sum','download_pk':'sum'}).collect()




In [ ]:
flow_time_series_df = pd.DataFrame(flow_time_series)
flow_time_series_df.columns = ['time','mean_duration','pk_count','byte_count','upload_bytes','download_bytes','upload_pk','download_pk']


In [ ]:
red_flow_time_series_df = pd.DataFrame(red_flow_time_series)
red_flow_time_series_df.columns = ['time','mean_duration','pk_count','byte_count','upload_bytes','download_bytes','upload_pk','download_pk']


In [ ]:
plt.plot(flow_time_series_df['time'],flow_time_series_df['upload_bytes'],label = 'Upload')
plt.plot(flow_time_series_df['time'],flow_time_series_df['download_bytes'],label = 'Download')
plt.legend()

In [ ]:
plt.plot(flow_time_series_df['time'],flow_time_series_df['download_bytes']/flow_time_series_df['upload_bytes'])




In [ ]:
plt.plot(red_flow_time_series_df['time'],red_flow_time_series_df['upload_bytes'],label = 'Upload')
plt.plot(red_flow_time_series_df['time'],red_flow_time_series_df['download_bytes'],label = 'Download')

plt.legend()

In [ ]:
plt.plot(red_flow_time_series_df['time'],red_flow_time_series_df['download_bytes']/red_flow_time_series_df['upload_bytes'])



